# Intro

For this project, we will be exploring the [World Factbook](https://www.cia.gov/the-world-factbook/) published by the United States Central Intelligence Agency. It provides a variety of information on 267 world entities. We will use basic SQL queries to answer interesting questions about the world.

# Pre-code

The following code will connect Jupyter Notebook to the database `factbook.db`.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

The following query provides information on the tables in the database.

In [2]:
%%sql

SELECT *
  FROM sqlite_master
 WHERE type = 'table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Data overview

Let's take a look at the first 5 rows.

In [3]:
%%sql

SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- `name` - the name of the country
- `area` - the total land and sea area of the country in square kilometers
- `area_land` - the country's land area in square kilometers
- `area_water` - the country's water area in square kilometers
- `population` - the country's population
- `population_growth` - the country's population growth as a percentage
- `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people
- `death_rate` - the country's death rate, or the number of deaths per year per 1,000 people

# Summary statistics

Let's start by calculating some summary statistics and see what they tell us.

In [4]:
%%sql

SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Which country has the minimum population of 0?

In [5]:
%%sql

SELECT name
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts);

Done.


name
Antarctica


Which country has the maximum population of almost 7.26 billion?

In [6]:
%%sql

SELECT name
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts);

Done.


name
World


Ah, the facts table actually contains a row corresponding to the entire world.

# Summary statistics re-visited

Let's re-compute the summary statistics while excluding the rows pertaining to `World` and `Antarctica`.

In [7]:
%%sql

SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name != 'World'
   AND name != 'Antarctica';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


Now that we've excluded `World` and `Antarctica`, let's find out which countries have the maximum population of almost 1.4 billion and the minimum population of 48.

# Country with maximum population

In [8]:
%%sql

SELECT *
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                      WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


# Country with minimum population

In [9]:
%%sql

SELECT *
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts
                      WHERE name != 'Antarctica');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


# Top 10 and bottom 10 countries by population

Let's see which countries were just shy of being named most or least populated. First, the most populated:

In [10]:
%%sql

SELECT name, population
  FROM facts
 WHERE name != 'World'
 ORDER BY population DESC
 LIMIT 10;

Done.


name,population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773


And now, the least populated:

In [11]:
%%sql

SELECT name, population
  FROM facts
 WHERE name != 'Antarctica'
   AND population IS NOT NULL
 ORDER BY population
 LIMIT 10;

Done.


name,population
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190
Tokelau,1337
Christmas Island,1530
Svalbard,1872
Norfolk Island,2210
Falkland Islands (Islas Malvinas),3361
Montserrat,5241


# Average population and average area

Let's compute the average population and average area of all countries.

In [12]:
%%sql

SELECT ROUND(AVG(population)) AS avg_pop,
       ROUND(AVG(area)) AS avg_area
  FROM facts
 WHERE name != 'World';

Done.


avg_pop,avg_area
32242667.0,555094.0


# Densely populated countries

In order to identify densely populated countries, let's find all countries with above-average `population` and below-average `area`.

In [13]:
%%sql

SELECT name
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != 'World')
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name != 'World');

Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


Let's find densely populated countries by using `population` / `area` directly.

In [14]:
%%sql

SELECT name, population / area AS pop_density
  FROM facts
 ORDER BY pop_density DESC
 LIMIT 10;

Done.


name,pop_density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


# Which countries will add the most people to their population next year?

In [15]:
%%sql

SELECT name, ROUND(population * population_growth) AS annual_people_added
  FROM facts
 WHERE name != 'World'
 ORDER BY annual_people_added DESC
 LIMIT 10;

Done.


name,annual_people_added
India,1527068612.0
China,615368425.0
Nigeria,444827037.0
Pakistan,290665337.0
Ethiopia,287456217.0
Bangladesh,270332392.0
United States,250667714.0
Indonesia,235514180.0
"Congo, Democratic Republic of the",194469083.0
Philippines,162607385.0


# Are there any countries with more water than land?

Let's query the database for water to land ratio and order the results accordingly. A water to land ratio above 1.0 means more water than land. So let's see if any fit the bill.

In [16]:
%%sql

SELECT name, area_water, area_land,
       ROUND(1.0 * area_water / area_land, 2) AS W2L_ratio
  FROM facts
 ORDER BY W2L_ratio DESC
 LIMIT 5;

Done.


name,area_water,area_land,W2L_ratio
British Indian Ocean Territory,54340,60,905.67
Virgin Islands,1564,346,4.52
Puerto Rico,4921,8870,0.55
"Bahamas, The",3870,10010,0.39
Guinea-Bissau,8005,28120,0.28


Result number 1 with a whopping water to land ratio just over 900 is the British Indian Ocean Territory. Let's exclude this answer as an outlier since it is a territory in the Indian Ocean with many very small islands -- not a country in the typical sense. Thus, result number 2, the Virgin Islands actually has more water than land -- approximately 4.5 times as much!

# Which countries have a higher death rate than birth rate?

In [17]:
%%sql

SELECT name, birth_rate, death_rate
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY death_rate / birth_rate DESC;

Done.


name,birth_rate,death_rate
Bulgaria,8.92,14.44
Serbia,9.08,13.66
Latvia,10.0,14.31
Lithuania,10.1,14.27
Hungary,9.16,12.73
Monaco,6.65,9.24
Slovenia,8.42,11.37
Ukraine,10.72,14.46
Germany,8.47,11.42
Saint Pierre and Miquelon,7.42,9.72
